# Reportagem: Execução orçamentária com Cultura pela Prefeitura de Fortaleza entre 2010 a 2022

Este repositório reúne códigos construídos para coleta e análise de dados para a realização da reportagem "[Cultura na gestão Sarto é marcada por recordes negativos de investimentos na área](https://mais.opovo.com.br/reportagens-especiais/2023/04/10/cultura-na-gestao-sarto-e-marcada-por-recordes-negativos-de-investimentos-na-area.html)", publicada no [O POVO+](https://mais.opovo.com.br/) em 10 de abril de 2023 e produzida em parceria com o Vida & Arte, núcleo de Cultura do O POVO.

**ÍNDICE**

1. [Coleta, limpeza e checagem dos dados](#coleta-limpeza-checagem)

    1.1 [Despesas por função](#despesas-funcao)

    1.2 [Despesas por área de atuação](#despesas-area-atuacao)


2. [Análise e visualizações](#analise-e-visualizacoes)

    2.1 [Valor empenhado para a Cultura por ano em Fortaleza-CE, entre 2010 e 2022 e Percentual do valor do orçamento empenhado para o setor cultural em relação às demais funções de governo entre 2010 e 2022](#valor-empenhado-demais-funcoes)

    2.2 [Valor empenhado em Cultura em cada ano (2010-2022) em relação ao que estava previsto na Lei Orçamentária Anual (LOA)](#valor-empenhado-loa)

    2.3 [Valores Empenhados, Liquidados e Pagos para a Cultura, entre 2014 e 2022](#valores-empenhados-liquidados-pagos)

## 1. Coleta, limpeza e checagem dos dados <a name="coleta-limpeza-checagem"></a>

In [1]:
import os
import pandas as pd
import numpy as np

### 1.1 Despesas por função <a name="despesas-funcao"></a>

O código utilizado para coleta, via *Selenium*, está disponível abaixo como comentário. Neste arquivo apenas realizamos a leitura e tratamento dos dados previamente coletados. 

In [2]:
arquivos = 'arquivos/'


Código utilizado para coleta (para saber mais sobre as configurações da biblioteca utilizada, consulte [este link](https://pypi.org/project/selenium/): 


``` python 
import pandas as pd

from selenium.webdriver import Chrome, ChromeOptions, Firefox
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import time 
from selenium.webdriver.firefox.options import Options


for a in range(2010,2023):
    print(a)
    meses = ['dez', 'nov', 'out', 'set', 'ago', 'jul', 'jun', 'mai', 'abr','mar', 'fev', 'jan']
    colunas = ['descrição da despesa','Valor Anual Previsto na LOA','valor mensal empenhado','Valor Acumulado']
    df_ano_mes = []
    for i,m in enumerate(meses):
        print(m)
        url = f'https://transparencia.fortaleza.ce.gov.br/index.php/despesa/exibirDespesasPorFuncao/{a}#tabs-{m}'

        options = Options()
        options.headless = True
        webdriver = Firefox(executable_path='/home/alexandre/bin/geckodriver', options=options)

        webdriver.get(url)
        time.sleep(3)

        table = webdriver.find_elements(By.CLASS_NAME, 'table')[11-i]
        
        rows = table.find_elements(By.TAG_NAME, "tr") 
        cr = 0
        lista_dados = []
        for row in rows[1:]:
            col = row.find_elements(By.TAG_NAME, "td")
            lista_dados.append({
                                'ano':a,
                                'mes':m,
                                colunas[0]:col[0].text,
                                colunas[1]:col[1].text,
                                colunas[2]:col[2].text,
                                colunas[3]:col[3].text,
                                })
        df = pd.DataFrame(lista_dados)

        df_ano_mes.append(df)

        df = pd.concat(df_ano_mes)
        df.to_csv(arquivos + f'df_{a}.csv')
        webdriver.close()
        
```




In [3]:
dict_meses = {'dez':12,
              'nov':11, 
              'out':10,
              'set':9, 
              'ago':8, 
              'jul':7, 
              'jun':6,
              'mai':5,
              'abr':4,
              'mar':3,
              'fev':2,
              'jan':1
             }

In [4]:
pd.options.display.float_format = '{:.2f}'.format

In [5]:
!pip install calculadora_do_cidadao

You should consider upgrading via the '/home/alexandre/anaconda3/envs/credos/bin/python -m pip install --upgrade pip' command.


In [6]:
from calculadora_do_cidadao import Ipca
from datetime import date

In [7]:
ipca = Ipca()

In [8]:
path_scrap = 'web_scrap_selenium'

In [9]:
lista_df = []

for f in os.listdir(path_scrap):
    lista_df.append(pd.read_csv(path_scrap+'/'+f))

wsDf = pd.concat(lista_df)[['ano', 'mes', 'descrição da despesa',
       'Valor Anual Previsto na LOA', 'valor mensal empenhado',
       'Valor Acumulado']]

In [10]:
wsDf['ano/mes'] = wsDf.apply(lambda x: f'{x.mes}/{x.ano}', axis=1)

In [11]:
wsDf['Valor Anual Previsto na LOA'] = wsDf['Valor Anual Previsto na LOA'].str.replace('.', '').str.replace(',', '.').astype(float)
wsDf['valor mensal empenhado'] = wsDf['valor mensal empenhado'].str.replace('.', '').str.replace(',', '.').astype(float)
wsDf['Valor Acumulado'] = wsDf['Valor Acumulado'].str.replace('.', '').str.replace(',', '.').astype(float)

/home/alexandre/anaconda3/envs/credos/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.
/home/alexandre/anaconda3/envs/credos/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  
/home/alexandre/anaconda3/envs/credos/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imp

In [12]:
wsDf['dtg'] = wsDf.apply(lambda x: pd.to_datetime(f'{x.ano}-{dict_meses[x.mes]}'), axis=1)

In [13]:
wsDf['valor mensal empenhado corrigido'] = wsDf.apply(lambda x: x['valor mensal empenhado'] * float(ipca.adjust(x['dtg'])), axis=1)
wsDf['valor Acumulado corrigido'] = wsDf.apply(lambda x: x['Valor Acumulado'] * float(ipca.adjust(x['dtg'])), axis=1)

In [14]:
wsDf.loc[wsDf['descrição da despesa'] != 'TOTAIS'][['ano','mes','valor mensal empenhado','valor mensal empenhado corrigido','dtg']]

,ano,mes,valor mensal empenhado,valor mensal empenhado corrigido,dtg
0,2011,dez,5517592.09,10714558.12,2011-12-01
1,2011,dez,3576478.19,6945128.02,2011-12-01
2,2011,dez,43383210.84,84245432.86,2011-12-01
3,2011,dez,6732063.62,13072928.51,2011-12-01
4,2011,dez,5726340.89,11119925.37,2011-12-01
...,...,...,...,...,...
294,2018,jan,302250.00,405168.57,2018-01-01
295,2018,jan,0.00,0.00,2018-01-01
296,2018,jan,2789574.00,3739446.49,2018-01-01
297,2018,jan,10738736.95,14395363.65,2018-01-01


### 1.2 Despesas por área de atuação <a name="despesas-area-atuacao"></a>

A coleta dos dados foi feita com o webscraper, extensão para browsers. Para saber mais sobre a ferramenta, acesse [este link](https://www.webscraper.io/).

In [15]:
df = pd.read_csv('arquivos/despesas_por_area_de_atuacao.csv')

In [16]:
funcoes = ['ADMINISTRACAO', 'ASSISTENCIA SOCIAL', 'CIENCIA E TECNOLOGIA', 'COMERCIO E SERVICOS', 'COMUNICACOES',  'CULTURA', 'DESPORTO E LAZER', 'DIREITOS DA CIDADANIA', 'EDUCACAO', 'ENCARGOS ESPECIAIS', 'ENERGIA', 'ESSENCIAL A JUSTICA', 'GESTAO AMBIENTAL', 'HABITACAO', 'JUDICIARIA', 'LEGISLATIVA', 'PREVIDENCIA SOCIAL', 'RESERVA DE CONTINGENCIA', 'SANEAMENTO', 'SAUDE', 'SEGURANCA PUBLICA', 'TRABALHO', 'TRANSPORTE', 'URBANISMO']

In [17]:
df['tipo'] = np.where(df['função / subfunção'].isin(funcoes), 'função', 'subfunção')

In [18]:
df['ano'] = df['web-scraper-start-url'].apply(lambda x: x.split("exercicio=")[1].split("&")[0]) 

In [19]:
df['Dotação Orçamentária'] = df['Dotação Orçamentária'].str.replace('.', '').str.replace(',', '.').astype(float)
df['Valor Empenhado'] = df['Valor Empenhado'].str.replace('.', '').str.replace(',', '.').astype(float)
df['Valor Liquidado'] = df['Valor Liquidado'].str.replace('.', '').str.replace(',', '.').astype(float)
df['Valor Pago'] = df['Valor Pago'].str.replace('.', '').str.replace(',', '.').astype(float)

/home/alexandre/anaconda3/envs/credos/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.
/home/alexandre/anaconda3/envs/credos/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  
/home/alexandre/anaconda3/envs/credos/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imp

In [20]:
df['dtg'] = df.ano.apply(lambda x: pd.to_datetime(f'{x}-12-31')) 

In [21]:
df['valor_pago_corrigido'] = df.apply(lambda x: x['Valor Pago'] * float(ipca.adjust(x['dtg'])), axis=1)
df['valor_empenhado_corrigido'] = df.apply(lambda x: x['Valor Empenhado'] * float(ipca.adjust(x['dtg'])), axis=1)
df['valor_liquidado_corrigido'] = df.apply(lambda x: x['Valor Liquidado'] * float(ipca.adjust(x['dtg'])), axis=1)

In [22]:
def getSubfunc(idx_ano):
    cont = idx_ano+1
    while df.iloc[cont].tipo == 'subfunção':
        cont+=1
        return df.iloc[idx_ano+1:cont].copy()

In [23]:
lista_subfuncoes = []
for y in list(df[df['função / subfunção'] == 'CULTURA'].index):
    lista_subfuncoes.append(getSubfunc(y))

In [24]:
subfuncoes_cultura = pd.concat(lista_subfuncoes)

In [25]:
subfuncoes_cultura

,web-scraper-order,web-scraper-start-url,função / subfunção,Dotação Orçamentária,Valor Empenhado,%(B/A)%,Valor Liquidado,Valor Pago,tipo,ano,dtg,valor_pago_corrigido,valor_empenhado_corrigido,valor_liquidado_corrigido
71,1676496995-491,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,41801420.00,26512144.03,"63,42",20563025.45,19885172.83,subfunção,2022,2022-12-31,20301606.91,27067359.74,20993655.08
207,1676496998-627,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,23992607.00,18593628.52,"77,50",16668781.43,16373598.47,subfunção,2021,2021-12-31,17683558.05,20081200.22,18002356.94
347,1676497001-767,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,37927557.00,34162964.22,"90,07",33041567.19,32994905.57,subfunção,2020,2020-12-31,39219837.73,40608266.34,39275302.69
487,1676497004-907,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,43945423.00,40134986.66,"91,33",38109576.55,37869534.93,subfunção,2019,2019-12-31,47047625.38,49862133.79,47345843.68
630,1676497007-1050,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,17328743.00,15876914.38,"91,62",15729878.00,15689393.70,subfunção,2018,2018-12-31,20331237.80,20574238.11,20383699.74
767,1676497010-1187,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,29008519.00,27659035.17,"95,35",27437682.64,27380270.86,subfunção,2017,2017-12-31,36809931.31,37184700.98,36887115.49
890,1676497013-1310,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,36537654.00,27374169.02,"74,92",27008773.64,26971005.10,subfunção,2016,2016-12-31,37328442.59,37886430.00,37380715.05
1013,1676497016-1433,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,24532065.00,21236437.43,"86,57",21132017.44,20724053.97,subfunção,2015,2015-12-31,30486083.95,31239824.75,31086217.91
1145,1676497019-1565,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,30293643.00,17959184.64,"59,28",17959184.64,15888448.95,subfunção,2014,2014-12-31,25867247.73,29238516.58,29238516.58


## 2. Análise e visualizações <a name="analise-e-visualizacoes"></a>

### 2.1 Valor empenhado para a Cultura por ano em Fortaleza-CE, entre 2010 e 2022 e Percentual do valor do orçamento empenhado para o setor cultural em relação às demais funções de governo entre 2010 e 2022 <a name="valor-empenhado-demais-funcoes"></a>


In [26]:
resumo_cultura_total = wsDf.loc[(wsDf['descrição da despesa'] == 'TOTAIS') | (wsDf['descrição da despesa'] == 'CULTURA')].pivot(index=['ano','mes'], columns ='descrição da despesa', values = 'valor mensal empenhado corrigido').reset_index()

In [27]:
resumo_cultura_total['pct'] = round(resumo_cultura_total['CULTURA']/resumo_cultura_total['TOTAIS']*100,2)

In [28]:
resumo_cultura_total.columns = ['Ano','Mẽs', 'Cultura', 'Todas as funções', 'Pct'] 

In [29]:
resumo_cultura_total

,Ano,Mẽs,Cultura,Todas as funções,Pct
0,2010,abr,7073590.77,555995944.91,1.27
1,2010,ago,2906597.01,588668493.18,0.49
2,2010,dez,18982036.33,812105709.16,2.34
3,2010,fev,7104749.86,517809489.25,1.37
4,2010,jan,4030016.82,512144973.55,0.79
...,...,...,...,...,...
151,2022,mai,6251976.73,942592387.20,0.66
152,2022,mar,1449867.26,1134263322.26,0.13
153,2022,nov,17369496.43,1107236730.78,1.57
154,2022,out,3242717.96,916371918.97,0.35


In [30]:
resumo_cultura_total_ano =  wsDf.loc[((wsDf['descrição da despesa'] == 'TOTAIS') | (wsDf['descrição da despesa'] == 'CULTURA')) & (wsDf['mes'] == 'dez')].pivot(index='ano', columns ='descrição da despesa', values = 'valor Acumulado corrigido').reset_index()

In [31]:
resumo_cultura_total_ano['pct'] = round(resumo_cultura_total_ano['CULTURA']/resumo_cultura_total_ano['TOTAIS']*100,2)

In [32]:
resumo_cultura_total_ano.columns = ['Ano', 'Cultura', 'Todas as funções', 'Pct']

In [33]:
resumo_cultura_total_ano

,Ano,Cultura,Todas as funções,Pct
0,2010,64564116.30,6961418079.74,0.93
1,2011,81096228.21,7550418329.59,1.07
2,2012,92160883.08,8080775600.87,1.14
3,2013,63615872.43,8336238459.16,0.76
4,2014,72969625.50,8815842356.17,0.83
5,2015,47449827.62,8602436825.59,0.55
6,2016,64266493.68,8367582933.30,0.77
7,2017,58276549.16,8773069393.81,0.66
8,2018,77830169.75,8910916618.48,0.87
9,2019,89835038.43,10196634888.05,0.88


In [34]:
resumo_cultura_total_ano_sem_correcao =  wsDf.loc[((wsDf['descrição da despesa'] == 'TOTAIS') | (wsDf['descrição da despesa'] == 'CULTURA')) & (wsDf['mes'] == 'dez')].pivot(index='ano', columns ='descrição da despesa', values = 'Valor Acumulado').reset_index()

In [35]:
resumo_cultura_total_ano_sem_correcao['pct'] = round(resumo_cultura_total_ano_sem_correcao['CULTURA']/resumo_cultura_total_ano_sem_correcao['TOTAIS']*100,2)
resumo_cultura_total_ano_sem_correcao.columns = ['Ano', 'Cultura _ Sem Correção', 'Todas as funções _ Sem correção', 'Pct  _ Sem correção']

In [36]:
resumo_cultura_total_ano_sem_correcao

,Ano,Cultura _ Sem Correção,Todas as funções _ Sem correção,Pct _ Sem correção
0,2010,31217869.22,3365966292.85,0.93
1,2011,41761489.58,3888179800.35,1.07
2,2012,50230328.42,4404254807.14,1.14
3,2013,36721858.05,4812040669.91,0.76
4,2014,44820159.52,5414958046.03,0.83
5,2015,32255792.19,5847827663.35,0.55
6,2016,46434616.84,6045848856.99,0.77
7,2017,43347750.02,6525657824.35,0.66
8,2018,60060690.19,6876456829.67,0.87
9,2019,72309943.34,8207466751.46,0.88


In [37]:
pd.merge(resumo_cultura_total_ano_sem_correcao, resumo_cultura_total_ano, on='Ano')

,Ano,Cultura _ Sem Correção,Todas as funções _ Sem correção,Pct _ Sem correção,Cultura,Todas as funções,Pct
0,2010,31217869.22,3365966292.85,0.93,64564116.30,6961418079.74,0.93
1,2011,41761489.58,3888179800.35,1.07,81096228.21,7550418329.59,1.07
2,2012,50230328.42,4404254807.14,1.14,92160883.08,8080775600.87,1.14
3,2013,36721858.05,4812040669.91,0.76,63615872.43,8336238459.16,0.76
4,2014,44820159.52,5414958046.03,0.83,72969625.50,8815842356.17,0.83
5,2015,32255792.19,5847827663.35,0.55,47449827.62,8602436825.59,0.55
6,2016,46434616.84,6045848856.99,0.77,64266493.68,8367582933.30,0.77
7,2017,43347750.02,6525657824.35,0.66,58276549.16,8773069393.81,0.66
8,2018,60060690.19,6876456829.67,0.87,77830169.75,8910916618.48,0.87
9,2019,72309943.34,8207466751.46,0.88,89835038.43,10196634888.05,0.88


### 2.2 Valor empenhado em Cultura em cada ano (2010-2022) em relação ao que estava previsto na Lei Orçamentária Anual (LOA) <a name="valor-empenhado-loa"></a>

In [38]:
cultura_por_ano = wsDf.loc[(wsDf['descrição da despesa'] == 'CULTURA') &(wsDf['mes']== 'dez')].sort_values(by='ano')
cultura_por_ano['pct']  = round(cultura_por_ano['Valor Acumulado']/cultura_por_ano['Valor Anual Previsto na LOA']*100,2)
cultura_por_ano=cultura_por_ano[['ano','Valor Anual Previsto na LOA','Valor Acumulado','pct']]
cultura_por_ano.columns = ['Ano','Valor previsto na LOA','Valor empenhado no ano','Pct']
cultura_por_ano.to_csv(arquivos + 'valor_anual_previsto_LOA_e_valor_acumulado.csv', index=False)
cultura_por_ano

,Ano,Valor previsto na LOA,Valor empenhado no ano,Pct
9,2010,63770546.00,31217869.22,48.95
9,2011,78348304.00,41761489.58,53.30
9,2012,99861612.00,50230328.42,50.30
9,2013,90266544.00,36721858.05,40.68
11,2014,57645418.00,44820159.52,77.75
10,2015,57985508.00,32255792.19,55.63
10,2016,88398641.00,46434616.84,52.53
10,2017,96365773.00,43347750.02,44.98
10,2018,95069137.00,60060690.19,63.18
10,2019,104948694.00,72309943.34,68.90


### 2.3 Valores Empenhados, Liquidados e Pagos para a Cultura, entre 2014 e 2022 <a name="valores-empenhados-liquidados-pagos"></a>
(Valores totais e distribuídos por subfunção)


In [39]:
valores_cultura_soma = subfuncoes_cultura.groupby('ano').sum().round(2)
valores_cultura_soma

,Dotação Orçamentária,Valor Empenhado,Valor Liquidado,Valor Pago,valor_pago_corrigido,valor_empenhado_corrigido,valor_liquidado_corrigido
ano,,,,,,,
2014,30293643.00,17959184.64,17959184.64,15888448.95,25867247.73,29238516.58,29238516.58
2015,24532065.00,21236437.43,21132017.44,20724053.97,30486083.95,31239824.75,31086217.91
2016,36537654.00,27374169.02,27008773.64,26971005.10,37328442.59,37886430.00,37380715.05
2017,29008519.00,27659035.17,27437682.64,27380270.86,36809931.31,37184700.98,36887115.49
2018,17328743.00,15876914.38,15729878.00,15689393.70,20331237.80,20574238.11,20383699.74
2019,43945423.00,40134986.66,38109576.55,37869534.93,47047625.38,49862133.79,47345843.68
2020,37927557.00,34162964.22,33041567.19,32994905.57,39219837.73,40608266.34,39275302.69
2021,23992607.00,18593628.52,16668781.43,16373598.47,17683558.05,20081200.22,18002356.94
2022,41801420.00,26512144.03,20563025.45,19885172.83,20301606.91,27067359.74,20993655.08


In [40]:
valores_cultura = df[df['função / subfunção'] == 'CULTURA'][['ano','Dotação Orçamentária', 'Valor Empenhado','Valor Liquidado',
       'Valor Pago','%(B/A)%','valor_pago_corrigido','valor_empenhado_corrigido','valor_liquidado_corrigido']].sort_values(by='ano')
valores_cultura

,ano,Dotação Orçamentária,Valor Empenhado,Valor Liquidado,Valor Pago,%(B/A)%,valor_pago_corrigido,valor_empenhado_corrigido,valor_liquidado_corrigido
1144,2014,73248537.00,44820159.52,42945440.26,37758174.45,"61,19",61472334.74,72969625.50,69917481.91
1012,2015,55866508.00,32255792.19,32054672.20,29636524.95,"57,74",43596759.05,47449827.62,47153970.40
889,2016,90139716.00,46434616.84,44820208.66,43775480.12,"51,51",60586192.12,64266493.68,62032118.55
766,2017,79607923.00,43347750.02,41662131.35,41183168.28,"54,45",55366493.75,58276549.16,56010409.87
629,2018,84024373.00,60060690.19,58453363.52,58239392.83,"71,48",75470025.66,77830169.75,75747301.45
486,2019,90629635.00,72309943.34,68492245.07,68135466.38,"79,79",84648831.93,89835038.43,85092079.78
346,2020,172910763.00,136001177.20,131205471.39,131051846.32,"78,65",155776537.57,161659626.21,155959146.08
206,2021,100042367.00,83328486.20,75203332.04,73335155.17,"83,29",79202288.72,89995130.00,81219927.92
70,2022,98455818.00,42097589.76,36148471.18,34127215.29,"42,76",34841905.36,42979194.93,36905490.27


In [41]:
# Teste de valores totais para cultura e divididos por subfunção, a cada ano:

valores_cultura_soma.reset_index()['Valor Pago'] == valores_cultura.reset_index()['Valor Pago']

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
Name: Valor Pago, dtype: bool

In [42]:
valores_cultura_soma.reset_index()['Valor Liquidado'] == valores_cultura.reset_index()['Valor Liquidado']

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
Name: Valor Liquidado, dtype: bool

In [43]:
valores_cultura_soma.reset_index()['Valor Empenhado'] == valores_cultura.reset_index()['Valor Empenhado']

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
Name: Valor Empenhado, dtype: bool

In [44]:
valores_cultura_soma.reset_index()['Valor Pago'] == valores_cultura.reset_index()['Valor Pago']

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
Name: Valor Pago, dtype: bool

In [45]:
colunas = ['ano','Dotação Orçamentária', 'Valor Empenhado', 'Valor Liquidado', 'Valor Pago','função / subfunção','valor_pago_corrigido','valor_empenhado_corrigido','valor_liquidado_corrigido']

todas = df[df['função / subfunção'] == 'CULTURA'][colunas].copy().sort_values(by='ano')
todas['função / subfunção'] = 'Todas'

pd.concat([todas,subfuncoes_cultura[colunas]]).to_csv(arquivos + 'gastos_cultura_por_subfuncao.csv', index=False)


pd.concat([todas,subfuncoes_cultura[colunas]])

,ano,Dotação Orçamentária,Valor Empenhado,Valor Liquidado,Valor Pago,função / subfunção,valor_pago_corrigido,valor_empenhado_corrigido,valor_liquidado_corrigido
1144,2014,73248537.00,44820159.52,42945440.26,37758174.45,Todas,61472334.74,72969625.50,69917481.91
1012,2015,55866508.00,32255792.19,32054672.20,29636524.95,Todas,43596759.05,47449827.62,47153970.40
889,2016,90139716.00,46434616.84,44820208.66,43775480.12,Todas,60586192.12,64266493.68,62032118.55
766,2017,79607923.00,43347750.02,41662131.35,41183168.28,Todas,55366493.75,58276549.16,56010409.87
629,2018,84024373.00,60060690.19,58453363.52,58239392.83,Todas,75470025.66,77830169.75,75747301.45
486,2019,90629635.00,72309943.34,68492245.07,68135466.38,Todas,84648831.93,89835038.43,85092079.78
346,2020,172910763.00,136001177.20,131205471.39,131051846.32,Todas,155776537.57,161659626.21,155959146.08
206,2021,100042367.00,83328486.20,75203332.04,73335155.17,Todas,79202288.72,89995130.00,81219927.92
70,2022,98455818.00,42097589.76,36148471.18,34127215.29,Todas,34841905.36,42979194.93,36905490.27
71,2022,41801420.00,26512144.03,20563025.45,19885172.83,ADMINISTRACAO GERAL,20301606.91,27067359.74,20993655.08


In [46]:
resumo_por_subfuncao =  subfuncoes_cultura[colunas].copy()

resumo_por_subfuncao.loc[(resumo_por_subfuncao['função / subfunção'] != 'ADMINISTRACAO GERAL') & (resumo_por_subfuncao['função / subfunção'] != 'DIFUSAO CULTURAL'),'função / subfunção'] = 'Outras'

resumo_por_subfuncao = resumo_por_subfuncao.groupby(['ano','função / subfunção']).sum().reset_index().pivot(index='ano', columns='função / subfunção', values='valor_pago_corrigido').fillna(0).reset_index()

In [47]:
resumo_por_subfuncao

função / subfunção,ano,ADMINISTRACAO GERAL
0,2014,25867247.73
1,2015,30486083.95
2,2016,37328442.59
3,2017,36809931.31
4,2018,20331237.80
5,2019,47047625.38
6,2020,39219837.73
7,2021,17683558.05
8,2022,20301606.91


In [48]:
resumo_por_subfuncao.to_csv(arquivos + 'resumo_por_subfuncao.csv', index=False)

In [49]:
resumo_por_subfuncao_v2 =  subfuncoes_cultura[colunas].copy()

resumo_por_subfuncao_v2 = resumo_por_subfuncao_v2.groupby(['ano','função / subfunção']).sum().reset_index().pivot(index='ano', columns='função / subfunção', values='valor_pago_corrigido').fillna(0).reset_index()

In [50]:
resumo_por_subfuncao_v2.to_csv(arquivos + 'resumo_por_subfuncao_v2.csv', index=False)

In [51]:
resumo_por_subfuncao_v2

função / subfunção,ano,ADMINISTRACAO GERAL
0,2014,25867247.73
1,2015,30486083.95
2,2016,37328442.59
3,2017,36809931.31
4,2018,20331237.80
5,2019,47047625.38
6,2020,39219837.73
7,2021,17683558.05
8,2022,20301606.91
